## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Importing Libraries</p>

In [39]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
# import seaborn as sns
# import scipy
# from mlxtend.classifier import StackingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from scipy.stats import mode
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import (
    roc_auc_score, roc_curve, auc, classification_report, 
    confusion_matrix, accuracy_score, matthews_corrcoef, f1_score,
    ConfusionMatrixDisplay
)
from sklearn.model_selection import (
    train_test_split, GridSearchCV, StratifiedShuffleSplit,
    StratifiedKFold, train_test_split, cross_val_score, cross_validate
)
import warnings
warnings.filterwarnings('ignore')

  ### <p style="background-color: #fdefff;color:#c12eff;display: inline-block;padding:.6rem;border-radius:.5rem;border: 1px solid #c059ff">Loading data</p>

In [40]:
cd /workspace/data/

/workspace/data


In [41]:
train_data = pd.read_csv(os.path.join(os.getcwd(), "kaggle/playground-series-s4e8/train.csv"))
test_data = pd.read_csv(os.path.join(os.getcwd(), "kaggle/playground-series-s4e8/test.csv"))
sample_submission_data = pd.read_csv(os.path.join(os.getcwd(), "kaggle/playground-series-s4e8/sample_submission.csv"))

print("train_data :", train_data.shape)
print("test_data :", test_data.shape)
print("sample_submission_data :", sample_submission_data.shape)

train_data : (3116945, 22)
test_data : (2077964, 21)
sample_submission_data : (2077964, 2)


## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Basic Info about Data</p> 

In [ ]:
train_data.head(2)

In [ ]:
test_data.head(2)

In [ ]:
sample_submission_data.head(2)

In [ ]:
train_data.info()

In [ ]:
train_data.describe().T

In [ ]:
print(train_data['class'].value_counts())
print(train_data['class'].value_counts() / sum(train_data['class'].value_counts()))
sns.countplot(x='class',data=train_data)
plt.xticks(rotation=60)
plt.show()

In [ ]:
pd.DataFrame({
    'column': train_data.columns,
    'null-count': train_data.isna().sum().values,
    '% null-count': np.round(train_data.isna().sum().values*100/len(train_data),6)
}).sort_values(by='null-count', ascending=False).reset_index(drop=True)

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Correlation Matrics</p> 

In [ ]:
# train_data.replace(['NaN', 'None', 'null', ''], np.nan, inplace=True)
df_dropped = train_data.drop('id', axis=1)
df_encoded = df_dropped.apply(lambda x: pd.factorize(x)[0] if x.dtype == 'object' else x)
train_data.isna().sum()

correlation_matrix = df_encoded.corr()

correlation_matrix
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Checking Unique Categories
</p> 

In [ ]:
# cate_col = train_data.select_dtypes(include=['object']).columns

# # Find unique categories and their counts for each categorical column
# unique_categories = {col: train_data[col].value_counts() for col in cate_col}

# # Set the size of the overall figure
# plt.figure(figsize=(15, len(cate_col) * 5))

# # Plot the count of each unique category
# for i, (col, counts) in enumerate(unique_categories.items(), 1):
#     plt.subplot(len(cate_col), 1, i)
#     sns.barplot(x=counts.index, y=counts.values, palette="viridis")
#     plt.title(f"Count of unique categories in column '{col}'")
#     plt.xlabel('Categories')
#     plt.ylabel('Count')
#     plt.xticks(rotation=45, ha='right')
#     plt.tight_layout()

# plt.show()


  ### <p style="background-color: #fdefff;color:#c12eff;display: inline-block;padding:.6rem;border-radius:.5rem;border: 1px solid #c059ff">Percentage of Missing Values by Feature</p>

In [ ]:
pd.DataFrame({
    'column': train_data.drop('id', axis=1).columns,
    'null-count': train_data.drop('id', axis=1).isna().sum().values,
    '% null-count': np.round(train_data.drop('id', axis=1).isna().sum().values*100/len(train_data.drop('id', axis=1)),6),
    'test null-count': test_data.isna().sum().values,
    '% test null-count': np.round(test_data.isna().sum().values*100/len(test_data),6)
}).sort_values(by='null-count', ascending=False).reset_index(drop=True)

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Checking Feature Importance and Dropping Useless Columns
</p> 

In [ ]:
# alpha = 0.05
# values = {}

# for col in train_data.columns:
#     if col == "class":
#         continue

#     A, B = train_data[col], train_data["class"]

#     dfObserved = pd.crosstab(A, B) 
#     chi2, p, dof, expected = scipy.stats.chi2_contingency(dfObserved.values) # 카이제곱 독립 검정 수행
#     values[col] = p
#     if p < alpha:
#         # Reject null hypothesis
#         print("{} is important. (p = {})".format(col, p))
#     else:
#         # Accept null hypothesis
#         print("{} is NOT important. (p = {})".format(col, p))

In [42]:
# Index(['id', 'class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
#        'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
#        'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
#        'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
#        'habitat', 'season', 'veil-info'],
#       dtype='object')

# 결측치와 특성 존재 여부를 동시에 다룰 수 있음: 앞에서 0과 1로 veil-type의 존재 여부를 표현하고, 뒤에서 'unknown'을 통해 veil-color의 결측치를 처리함으로써 두 정보를 효과적으로 결합할 수 있습니다.
# 모델 학습에 유용한 정보 제공: 결측치나 특성의 존재 여부를 단순히 무시하지 않고, 이들을 결합하여 모델이 더 많은 패턴을 학습할 수 있게 도와줍니다.
# Index(['w', 'y', 'n', 'u', 'k', 'e', 'g', 'p', 'r', 'o', 's', 'a', 't', 'd',
#        'i', 'h', 'c', 'f', 'l', 'b', 'z', '8.25', '2.49', '3.32'],
#       dtype='object', name='veil-color')
# Index(['u', 'w', 'a', 'f', 'e', 'b', 'c', 'y', 'k', 'g', 'n', 's', 'r', 'd',
#        'p', 'h', 'i', 'l', 'is None', 't', '21.11', '5.94'],
#       dtype='object', name='veil-type')

# Index(['f10', 'tnone', 't1', 't7', 't10', 't4', 't2', 't5', 't3', 't9',
#        'fnone', 'f1', 'f2', 'f7', 'f4', 't8', 'f5', 'r2', 't6', 'f3', 'l4',
#        't0', 'p5', 'z7', 'c10', 'x10', 'f9', 'f6', 's10', 'm9', 'hnone', 's1',
#        'g3', 'g5', 'h7', 'e1', 'f0', 'r4', 'dnone', 's5', 'cnone', 'h1', 'p1',
#        'h5', 'h10', 'w3', 'y2', 'a10', 'ynone', 'e10', 'p7', '10.310', 's2',
#        'o2', 'g10', 'h2', 'g1', 's3', 'p3', 'knone', 'inone', 'nnone', 'rnone',
#        'l5', 'c1', 'n10', 'c3', 'o10', 'e4', 'd10', 'f has-ring10', 'lnone',
#        'c7', 'e3', 'y1', 'k10'],
#       dtype='object', name='has-ring-type')

train_data['veil-info'] = train_data['veil-type'].notna().astype(int).astype(str) + train_data['veil-color'].fillna('unknown')
cap_shape_mapping = {'b': 0, 'c': 1, 'x': 2, 'f': 3, 's': 4, 'p': 5, 'o': 6}
cap_color_mapping = {'n': 0, 'b': 1, 'g': 2, 'r': 3, 'p': 4, 'u': 5, 'e': 6, 'w': 7, 'y': 8, 'l': 9, 'o': 10, 'k': 11}
ring_type_mapping = {'c': 0, 'e': 1, 'r': 2, 'g': 3, 'l': 4, 'p': 5, 's': 6, 'z': 7, 'y': 8, 'm': 9, 'f': 10}
train_data['cap-shape'] = train_data['cap-shape'].map(cap_shape_mapping).astype(str).str.replace('.0', '', regex=False)
train_data['cap-color'] = train_data['cap-color'].map(cap_color_mapping).astype(str).str.replace('.0', '', regex=False)
train_data['has-ring-type'] = (
    train_data['has-ring'] + 
    train_data['ring-type'].fillna('none').map(ring_type_mapping).apply(lambda x: f"{x:.0f}" if pd.notna(x) else 'none').str.replace('.0', '', regex=False)
)

test_data['veil-info'] = test_data['veil-type'].notna().astype(int).astype(str) + test_data['veil-color'].fillna('unknown')
test_data['cap-shape'] = test_data['cap-shape'].map(cap_shape_mapping).astype(str).str.replace('.0', '', regex=False)
test_data['cap-color'] = test_data['cap-color'].map(cap_color_mapping).astype(str).str.replace('.0', '', regex=False)
test_data['has-ring-type'] = (
    test_data['has-ring'] + 
    test_data['ring-type'].fillna('none').map(ring_type_mapping).apply(lambda x: f"{x:.0f}" if pd.notna(x) else 'none').str.replace('.0', '', regex=False)
)

In [5]:
train_data['has-ring-type']

0          f10
1           t7
2          f10
3          f10
4          f10
          ... 
3116940     t3
3116941    f10
3116942     t7
3116943     t5
3116944    f10
Name: has-ring-type, Length: 3116945, dtype: object

In [6]:
set(train_data['veil-info'])

{'02.49',
 '03.32',
 '08.25',
 '0a',
 '0b',
 '0c',
 '0d',
 '0e',
 '0f',
 '0g',
 '0h',
 '0i',
 '0k',
 '0l',
 '0n',
 '0o',
 '0p',
 '0r',
 '0s',
 '0t',
 '0u',
 '0unknown',
 '0w',
 '0y',
 '1g',
 '1h',
 '1k',
 '1n',
 '1r',
 '1t',
 '1u',
 '1unknown',
 '1w',
 '1y',
 '1z'}

In [43]:
train_data = train_data.drop(['id', 'veil-color', 'veil-type', 'has-ring', 'ring-type'], axis=1)
test_data = test_data.drop(['id', 'veil-color', 'veil-type', 'has-ring', 'ring-type'], axis=1)

In [44]:
# df_encoded = train_data
# dt_encoded = test_data

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Handling NaN Values And Less Frequent Categories</p> 

In [45]:
columns_to_fill_train = train_data.columns[(train_data.isna().sum() > 0) & (train_data.isna().sum() < 60)].tolist()
columns_to_fill_test = test_data.columns[(test_data.isna().sum() > 0) & (test_data.isna().sum() < 100)].tolist()

corr_matrix_train = df_encoded.corr()
corr_matrix_test = dt_encoded.corr()

def fill_missing_values(data, columns_to_fill, corr_matrix):
    for column in columns_to_fill:
        if column in corr_matrix.columns:
            most_corr_features = corr_matrix[column][corr_matrix[column].abs() > 0.1].index.tolist()
            if column in most_corr_features:
                most_corr_features.remove(column)

            if most_corr_features:
                group_modes = data.groupby(most_corr_features)[column].apply(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

                def fill_na(row):
                    if pd.isna(row[column]):
                        try:
                            return group_modes.loc[tuple(row[most_corr_features])]
                        except KeyError:
                            return np.nan
                    else:
                        return row[column]

                data[column] = data.apply(fill_na, axis=1)

fill_missing_values(df_encoded, columns_to_fill_train, corr_matrix_train)
fill_missing_values(dt_encoded, columns_to_fill_test, corr_matrix_test)


NameError: name 'df_encoded' is not defined

In [ ]:
train_data.isna().sum()

In [ ]:
train_data.columns[(train_data.isna().sum() > 0) & (train_data.isna().sum() < 60)].tolist()

In [52]:
def cleaning(df):
    threshold = 100
    
    cat_feats = ['cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-root', 'stem-surface', 'stem-color', 'spore-print-color',
       'habitat', 'season', 'veil-info', 'has-ring-type']
    
    for feat in cat_feats:
        if df[feat].dtype.name == 'category':
            # Add 'missing' and 'noise' to categories if not present
            if 'missing' not in df[feat].cat.categories:
                df[feat] = df[feat].cat.add_categories('missing')
            if 'noise' not in df[feat].cat.categories:
                df[feat] = df[feat].cat.add_categories('noise')
        else:
            # Convert to category and add new categories
            df[feat] = df[feat].astype('category')
            df[feat] = df[feat].cat.add_categories(['missing', 'noise'])
        
        # Fill missing values with 'missing'
        df[feat] = df[feat].fillna('missing')
        
        # Replace infrequent categories with 'noise'
        counts = df[feat].value_counts(dropna=False)
        infrequent_categories = counts[counts < threshold].index
        df[feat] = df[feat].apply(lambda x: 'missing' if x in infrequent_categories else x)
    
    return df

# Example usage
train_data = cleaning(train_data)
test_data = cleaning(test_data)

In [53]:
train_data.isna().sum()

class                   0
cap-diameter            4
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-root               0
stem-surface            0
stem-color              0
spore-print-color       0
habitat                 0
season                  0
veil-info               0
has-ring-type           0
dtype: int64

In [54]:
group_by_features = ['stem-width', 'stem-height']
group_means_train = train_data.groupby(group_by_features)['cap-diameter'].mean()
group_means_train

stem-width  stem-height
0.00        0.00            4.253239
            2.69            3.900000
            2.85            3.040000
            2.93            2.810000
            2.94            2.840000
                             ...    
102.48      26.04           9.910000
            26.38          20.490000
            26.40          27.080000
            26.44          23.710000
102.90      27.32          17.570000
Name: cap-diameter, Length: 1134552, dtype: float64

In [55]:
group_by_features = ['stem-width', 'stem-height']
group_means_train = train_data.groupby(group_by_features)['cap-diameter'].mean()

def fill_na_with_group_mean(row):
    if pd.isna(row['cap-diameter']):
        group = tuple(row[group_by_features])
        return group_means_train.get(group, np.nan) 
    else:
        return row['cap-diameter']

train_data['cap-diameter'] = train_data.apply(fill_na_with_group_mean, axis=1)
test_data['cap-diameter'] = test_data.apply(fill_na_with_group_mean, axis=1)

In [56]:
group_by_features = ['stem-width', 'stem-height']

# Calculate group means for the train data
group_means_train = train_data.groupby(group_by_features)['cap-diameter'].mean()

def fill_na_with_group_mean(row, group_means):
    if pd.isna(row['cap-diameter']):
        group = tuple(row[group_by_features])
        return group_means.get(group, np.nan)
    else:
        return row['cap-diameter']

# Apply to train_data using train group means
train_data['cap-diameter'] = train_data.apply(fill_na_with_group_mean, axis=1, group_means=group_means_train)

# Apply the same group means from train_data to test_data
test_data['cap-diameter'] = test_data.apply(fill_na_with_group_mean, axis=1, group_means=group_means_train)
###
###
# Calculate the mode from the training data
cap_diameter_mode = train_data['cap-diameter'].mode()[0]
stem_height_mode = train_data['stem-height'].mode()[0]

# Fill missing values in the training data using the mode calculated from the training data
train_data['cap-diameter'] = train_data['cap-diameter'].fillna(cap_diameter_mode)
train_data['stem-height'] = train_data['stem-height'].fillna(stem_height_mode)

# Fill missing values in the test data using the mode calculated from the training data
test_data['cap-diameter'] = test_data['cap-diameter'].fillna(cap_diameter_mode)
test_data['stem-height'] = test_data['stem-height'].fillna(stem_height_mode)

In [57]:
# print("Nan Values in Train data")
# print(null_percent(train_data))
# print("Nan Values in Test data")
# print(null_percent(test_data))

In [58]:
cat_feats = ['cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-root', 'stem-surface', 'stem-color', 'spore-print-color',
       'habitat', 'season', 'veil-info', 'has-ring-type']

for feat in cat_feats:
    train_data[feat] = train_data[feat].astype('category')
for feat in cat_feats:
    test_data[feat] = test_data[feat].astype('category')

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Splitting Data</p> 

In [59]:
X = train_data.drop(['class'], axis=1)
y = train_data['class']
print(X.shape, y.shape)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
print(y[:3])

(3116945, 18) (3116945,)
[0 1 0]


In [60]:
# Callback to print the MCC score for each trial
def print_mcc_callback(study, trial):
    mcc = trial.user_attrs["mcc"]
    print(f"Trial {trial.number}: MCC = {mcc}")

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Using XGBoost with optuna</p> 

In [ ]:
def model_report(estimator, X, y, cv=5):
    print("="*80)
    print(f"    Model: {estimator.__class__.__name__}")
    print("="*80)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=1/cv, shuffle=True, stratify=y, random_state=42)
    
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    print(f"F1 Score : {f1.mean():.6f}")
    print(f"MCC Score: {mcc.mean():.6f}")
    
    ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred)).plot()
    plt.title("Confusion Matrix")
    plt.show()

    print()

In [ ]:
xgb_clf = XGBClassifier(enable_categorical=True, device="cuda", tree_method="hist")

model_report(xgb_clf, X, y)

In [ ]:
cat_clf = CatBoostClassifier(
    cat_features=cat_feats,
    verbose=False,
    allow_writing_files=False,
    task_type="GPU"
)

model_report(cat_clf, X, y)

In [ ]:
lgb_clf = LGBMClassifier(device='cpu', verbosity=-1)

model_report(lgb_clf, X, y)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4885)

# Define the objective function for Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1200), #  100, 1000
        'max_depth': trial.suggest_int('max_depth', 10, 22), #5, 10
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0), # 0.5, 1.0
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 10.0),
        'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1.0, 10.0),
        "eta": trial.suggest_float("eta", 1e-3, 1e-2),
        "min_child_weight": trial.suggest_int("min_child_weight", 40, 100),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "max_leaves": trial.suggest_int("max_leaves", 40, 100), # 16, 84
        'device': 'cuda',
        'tree_method': 'gpu_hist'
    }

    model = XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss', enable_categorical=True, early_stopping_rounds=20)
    # Fit the model with early stopping
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],  # Validation set
        verbose=False                 # Suppress output
    )
    
    y_pred = model.predict(X_test)
    mcc = matthews_corrcoef(y_test, y_pred)
    trial.set_user_attr("mcc", mcc)
    return mcc



storage = "sqlite:///xgb.db"

# Optimize hyperparameters with Optuna
study = optuna.create_study(storage=storage,
                            direction='maximize',
                           )
study.optimize(objective, n_trials=100, callbacks=[print_mcc_callback])

# Get the best parameters
best_params = study.best_params
print(f"Best parameters: {best_params}")

[I 2024-08-28 04:15:50,045] A new study created in RDB with name: no-name-ca990677-b4c9-42bd-bc95-e730c74f599c
[I 2024-08-28 04:15:59,670] Trial 0 finished with value: 0.9776301615156926 and parameters: {'n_estimators': 1127, 'max_depth': 22, 'learning_rate': 0.0628120330364908, 'subsample': 0.7667437449993406, 'colsample_bytree': 0.58218155552719, 'gamma': 1.3067518264009852e-08, 'lambda': 0.24318162303993915, 'alpha': 1.2318547266246812e-06, 'scale_pos_weight': 8.181237871416652, 'eta': 0.009631693778132562, 'min_child_weight': 60, 'grow_policy': 'lossguide', 'max_leaves': 34}. Best is trial 0 with value: 0.9776301615156926.


Trial 0: MCC = 0.9776301615156926


[I 2024-08-28 04:16:03,914] Trial 1 finished with value: 0.9797799674518712 and parameters: {'n_estimators': 851, 'max_depth': 21, 'learning_rate': 0.0556835815532188, 'subsample': 0.5158707003741055, 'colsample_bytree': 0.9794851691527583, 'gamma': 0.003571690919366008, 'lambda': 0.024927352176999686, 'alpha': 0.06865882398910715, 'scale_pos_weight': 5.082528490600433, 'eta': 0.0023765259061155636, 'min_child_weight': 67, 'grow_policy': 'depthwise', 'max_leaves': 35}. Best is trial 1 with value: 0.9797799674518712.


Trial 1: MCC = 0.9797799674518712


[I 2024-08-28 04:16:08,772] Trial 2 finished with value: 0.9845368094428952 and parameters: {'n_estimators': 796, 'max_depth': 10, 'learning_rate': 0.09625493107435389, 'subsample': 0.8585019793154267, 'colsample_bytree': 0.8435427252703662, 'gamma': 5.8073553483094284e-08, 'lambda': 1.591999930585006e-07, 'alpha': 3.059604382853954e-05, 'scale_pos_weight': 1.8193163843481919, 'eta': 0.008163048300847301, 'min_child_weight': 46, 'grow_policy': 'depthwise', 'max_leaves': 83}. Best is trial 2 with value: 0.9845368094428952.


...

Trial 97: MCC = 0.981915949968389


[I 2024-08-28 04:24:21,233] Trial 98 finished with value: 0.9812094897608734 and parameters: {'n_estimators': 618, 'max_depth': 20, 'learning_rate': 0.11873906860607439, 'subsample': 0.989080524800864, 'colsample_bytree': 0.5714305756830748, 'gamma': 2.598287432874638e-07, 'lambda': 0.3580822878816201, 'alpha': 6.973646287001525e-06, 'scale_pos_weight': 5.013481983990574, 'eta': 0.006804789168684357, 'min_child_weight': 94, 'grow_policy': 'depthwise', 'max_leaves': 63}. Best is trial 16 with value: 0.984876834594928.


Trial 98: MCC = 0.9812094897608734


[I 2024-08-28 04:24:26,117] Trial 99 finished with value: 0.980280915856827 and parameters: {'n_estimators': 767, 'max_depth': 19, 'learning_rate': 0.09944696536135691, 'subsample': 0.8550101480132652, 'colsample_bytree': 0.5521070597958867, 'gamma': 1.4662637319697745e-06, 'lambda': 1.2857697148586766e-08, 'alpha': 0.6298241361049903, 'scale_pos_weight': 5.87662807485976, 'eta': 0.004908567506182798, 'min_child_weight': 73, 'grow_policy': 'depthwise', 'max_leaves': 67}. Best is trial 16 with value: 0.984876834594928.


Trial 99: MCC = 0.980280915856827
Best parameters: {'n_estimators': 629, 'max_depth': 16, 'learning_rate': 0.10540811331505591, 'subsample': 0.9980081491289166, 'colsample_bytree': 0.7065318052513923, 'gamma': 2.8307622020907574e-06, 'lambda': 0.02456026309353021, 'alpha': 1.5584362263554227e-07, 'scale_pos_weight': 1.105994872793953, 'eta': 0.0010400771543147627, 'min_child_weight': 79, 'grow_policy': 'depthwise', 'max_leaves': 61}


In [17]:
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        "random_strength": trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 10.0),
        "od_type": trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        "od_wait": trial.suggest_int("od_wait", 10, 50),
        "verbose": False,
        "allow_writing_files": False,
        "task_type": 'GPU',
        "cat_features": cat_feats
    }

    model = CatBoostClassifier(**params)

    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],  # Validation set
        early_stopping_rounds=20,
        verbose=False                 # Suppress output
    )
    y_pred = model.predict(X_test)
    mcc = matthews_corrcoef(y_test, y_pred)
    trial.set_user_attr("mcc", mcc)
    return mcc


study_name = "cat"
storage = "sqlite:///cat.db"

study = optuna.create_study(storage=storage,
                            study_name=study_name,
                            direction="maximize",
                            sampler=TPESampler(n_startup_trials=20, multivariate=True),
                            load_if_exists=True)

study.optimize(objective, n_trials=50, callbacks=[print_mcc_callback])

print(study.best_params)

[I 2024-08-28 04:24:26,184] A new study created in RDB with name: cat
[I 2024-08-28 04:27:02,105] Trial 0 finished with value: 0.9831942903018556 and parameters: {'iterations': 2889, 'learning_rate': 0.03224196574593359, 'depth': 6, 'l2_leaf_reg': 3.0230069060597975, 'bootstrap_type': 'Bayesian', 'random_strength': 4.611328675401496, 'bagging_temperature': 6.87634627407963, 'od_type': 'IncToDec', 'od_wait': 34}. Best is trial 0 with value: 0.9831942903018556.


Trial 0: MCC = 0.9831942903018556


[I 2024-08-28 04:28:22,528] Trial 1 finished with value: 0.9839282005852762 and parameters: {'iterations': 1708, 'learning_rate': 0.07960332055478367, 'depth': 5, 'l2_leaf_reg': 0.2282485542914049, 'bootstrap_type': 'Bayesian', 'random_strength': 1.4505005237747335e-05, 'bagging_temperature': 1.1292587646316243, 'od_type': 'Iter', 'od_wait': 16}. Best is trial 1 with value: 0.9839282005852762.


Trial 1: MCC = 0.9839282005852762


[I 2024-08-28 04:31:28,315] Trial 2 finished with value: 0.9832329005545981 and parameters: {'iterations': 2364, 'learning_rate': 0.017945586081204057, 'depth': 8, 'l2_leaf_reg': 3.750025236203084e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 0.008855170383120992, 'bagging_temperature': 9.602471956244825, 'od_type': 'IncToDec', 'od_wait': 42}. Best is trial 1 with value: 0.9839282005852762.


...

Trial 47: MCC = 0.9843623805113599


[I 2024-08-28 06:08:38,648] Trial 48 finished with value: 0.9843397500341847 and parameters: {'iterations': 2378, 'learning_rate': 0.016292819459672793, 'depth': 9, 'l2_leaf_reg': 5.78475211290315, 'bootstrap_type': 'Bayesian', 'random_strength': 2.2474019768028935e-06, 'bagging_temperature': 0.8485996098188662, 'od_type': 'IncToDec', 'od_wait': 26}. Best is trial 40 with value: 0.98473160011893.


Trial 48: MCC = 0.9843397500341847


[I 2024-08-28 06:09:52,330] Trial 49 finished with value: 0.9829442812722465 and parameters: {'iterations': 1967, 'learning_rate': 0.020351826742455875, 'depth': 8, 'l2_leaf_reg': 6.376879525946766e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 1.8259415944606363e-08, 'bagging_temperature': 4.60553501705422, 'od_type': 'Iter', 'od_wait': 14}. Best is trial 40 with value: 0.98473160011893.


Trial 49: MCC = 0.9829442812722465
{'iterations': 1472, 'learning_rate': 0.08436268683474936, 'depth': 8, 'l2_leaf_reg': 0.2621416701020154, 'bootstrap_type': 'Bayesian', 'random_strength': 4.490114031912514e-07, 'bagging_temperature': 0.42447506181344297, 'od_type': 'Iter', 'od_wait': 21}


In [18]:
def objective(trial):
    params = {
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "device": 'cpu',
        "verbosity": -1
    }

    model = LGBMClassifier(**params, early_stopping_rounds=20)

    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],  # Validation set
        # verbose=False                 # Suppress output
    )
    y_pred = model.predict(X_test)
    mcc = matthews_corrcoef(y_test, y_pred)
    trial.set_user_attr("mcc", mcc)
    return mcc


study_name = "lgb"
storage = "sqlite:///lgb.db"

study = optuna.create_study(storage=storage,
                            study_name=study_name,
                            direction="maximize",
                            sampler=TPESampler(n_startup_trials=20, multivariate=True),
                            load_if_exists=True)

study.optimize(objective, n_trials=100, callbacks=[print_mcc_callback])

print(study.best_params)

[I 2024-08-28 06:09:52,398] A new study created in RDB with name: lgb
[I 2024-08-28 06:09:56,674] Trial 0 finished with value: 0.9822336486882544 and parameters: {'lambda_l1': 2.790146194829057e-06, 'lambda_l2': 5.863566882191815, 'num_leaves': 41, 'feature_fraction': 0.8900436730638485, 'bagging_fraction': 0.6087325595294399, 'bagging_freq': 3, 'min_child_samples': 57}. Best is trial 0 with value: 0.9822336486882544.


Trial 0: MCC = 0.9822336486882544


[I 2024-08-28 06:10:02,030] Trial 1 finished with value: 0.9841649213216981 and parameters: {'lambda_l1': 4.608351741824684e-06, 'lambda_l2': 2.9842646024233264e-06, 'num_leaves': 132, 'feature_fraction': 0.72443783504399, 'bagging_fraction': 0.8834720151889617, 'bagging_freq': 3, 'min_child_samples': 58}. Best is trial 1 with value: 0.9841649213216981.


Trial 1: MCC = 0.9841649213216981


[I 2024-08-28 06:10:05,962] Trial 2 finished with value: 0.9839415848985409 and parameters: {'lambda_l1': 0.0030464686994775133, 'lambda_l2': 0.000409705083021584, 'num_leaves': 166, 'feature_fraction': 0.8264173391407263, 'bagging_fraction': 0.5125173275120677, 'bagging_freq': 6, 'min_child_samples': 21}. Best is trial 1 with value: 0.9841649213216981.


...

Trial 97: MCC = 0.984683483656782


[I 2024-08-28 06:17:55,584] Trial 98 finished with value: 0.9847645490060256 and parameters: {'lambda_l1': 2.401979835371133e-07, 'lambda_l2': 0.06718591525287267, 'num_leaves': 252, 'feature_fraction': 0.4079959103080681, 'bagging_fraction': 0.9758295868050206, 'bagging_freq': 1, 'min_child_samples': 97}. Best is trial 76 with value: 0.9848032057376603.


Trial 98: MCC = 0.9847645490060256


[I 2024-08-28 06:18:00,386] Trial 99 finished with value: 0.9847803908380041 and parameters: {'lambda_l1': 4.769505913779232e-07, 'lambda_l2': 0.021066558856617273, 'num_leaves': 256, 'feature_fraction': 0.45869814671689385, 'bagging_fraction': 0.8658416082514996, 'bagging_freq': 1, 'min_child_samples': 99}. Best is trial 76 with value: 0.9848032057376603.


Trial 99: MCC = 0.9847803908380041
{'lambda_l1': 1.3323938321586053e-07, 'lambda_l2': 0.07774032281247541, 'num_leaves': 248, 'feature_fraction': 0.41087472855265783, 'bagging_fraction': 0.9928696340850532, 'bagging_freq': 4, 'min_child_samples': 97}


In [48]:
# parameters={'n_estimators': 297, 'max_depth': 16, 'learning_rate': 0.03906159386409017, 'subsample': 0.6935900010487451, 'colsample_bytree': 0.5171160704967471, 'gamma': 0.00013710778966124443, 'lambda': 0.0017203271581656767, 'alpha': 8.501510750413265e-06, 'scale_pos_weight': 1.0017942891559255,'enable_categorical': True,'tree_method': 'hist', 'device': 'cuda'}

# 0.984876834594928
xgb_parameters={'n_estimators': 629, 'max_depth': 16, 'learning_rate': 0.10540811331505591, 'subsample': 0.9980081491289166, 'colsample_bytree': 0.7065318052513923, 'gamma': 2.8307622020907574e-06, 'lambda': 0.02456026309353021, 'alpha': 1.5584362263554227e-07, 'scale_pos_weight': 1.105994872793953, 'eta': 0.0010400771543147627, 'min_child_weight': 79, 'grow_policy': 'depthwise', 'max_leaves': 61,
               'device': 'cuda'
               }
# 0.98473160011893
cat_parameters={'iterations': 1472, 'learning_rate': 0.08436268683474936, 'depth': 8, 'l2_leaf_reg': 0.2621416701020154, 'bootstrap_type': 'Bayesian', 'random_strength': 4.490114031912514e-07, 'bagging_temperature': 0.42447506181344297, 'od_type': 'Iter', 'od_wait': 21,
                "task_type": 'GPU',
                "cat_features": cat_feats}
# 0.9848032057376603
lgb_parameters={'lambda_l1': 1.3323938321586053e-07, 'lambda_l2': 0.07774032281247541, 'num_leaves': 248, 'feature_fraction': 0.41087472855265783, 'bagging_fraction': 0.9928696340850532, 'bagging_freq': 4, 'min_child_samples': 97} 

In [49]:
def model_trainer(model, X, y, n_splits=5, random_state=4885):
    skfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    oof_probs, oof_mccs = [], []
    print("="*80)
    print(f"Training {model.__class__.__name__}")
    print("="*80, end="\n")
    for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
        X_train, y_train = X.iloc[train_idx, :], y[train_idx]
        X_test, y_test = X.iloc[test_idx, :], y[test_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mcc = matthews_corrcoef(y_pred, y_test)
        oof_mccs.append(mcc)
        oof_probs.append(model.predict_proba(test_data))
        print(f"--- Fold {fold+1} MCC Score: {mcc:.6f}")
    print(f"\n---> Mean MCC Score: {np.mean(oof_mccs):.6f} \xb1 {np.std(oof_mccs):.6f}\n\n")
    return oof_probs, oof_mccs

In [61]:
oof_probs = {}
oof_probs['xgb'], _ = model_trainer(XGBClassifier(**xgb_parameters, enable_categorical=True), X, y, random_state=4885)
oof_probs['cat'], _ = model_trainer(CatBoostClassifier(**cat_parameters), X, y, random_state=4885)
oof_probs['lgb'], _ = model_trainer(LGBMClassifier(**lgb_parameters), X, y, random_state=4885)

Training XGBClassifier
--- Fold 1 MCC Score: 0.984703
--- Fold 2 MCC Score: 0.984683
--- Fold 3 MCC Score: 0.984742
--- Fold 4 MCC Score: 0.984075
--- Fold 5 MCC Score: 0.984694

---> Mean MCC Score: 0.984579 ± 0.000253


Training CatBoostClassifier
0:	learn: 0.6011093	total: 72.5ms	remaining: 1m 46s
1:	learn: 0.5157252	total: 153ms	remaining: 1m 52s
2:	learn: 0.4469503	total: 217ms	remaining: 1m 46s
3:	learn: 0.3919998	total: 284ms	remaining: 1m 44s
4:	learn: 0.3465940	total: 354ms	remaining: 1m 43s
5:	learn: 0.3169362	total: 407ms	remaining: 1m 39s
6:	learn: 0.2809068	total: 476ms	remaining: 1m 39s
7:	learn: 0.2488664	total: 546ms	remaining: 1m 39s
8:	learn: 0.2369448	total: 608ms	remaining: 1m 38s
9:	learn: 0.2147048	total: 676ms	remaining: 1m 38s
10:	learn: 0.1940738	total: 749ms	remaining: 1m 39s
11:	learn: 0.1852591	total: 823ms	remaining: 1m 40s
12:	learn: 0.1760761	total: 876ms	remaining: 1m 38s
13:	learn: 0.1617247	total: 959ms	remaining: 1m 39s
14:	learn: 0.1515819	total: 1.0

In [62]:
oof_preds = {}
for model in oof_probs.keys():
    oof_preds[model] = np.argmax(np.mean(oof_probs[model], axis=0), axis=1)

In [63]:
np.argmax(np.mean(oof_probs['xgb'], axis=0), axis=1)[0]

0

In [64]:
np.mean(oof_probs['xgb'], axis=0)[0]

array([0.9951358 , 0.00486417], dtype=float32)

In [65]:
sub = pd.read_csv("./kaggle/playground-series-s4e8/sample_submission.csv")
preds = [pred for model, pred in oof_preds.items()]
md = mode(preds, axis=0)[0] if len(preds)>1 else preds[0]
sub['class'] = label_encoder.inverse_transform(md)
sub.to_csv('./kaggle/playground-series-s4e8/submission_xgb_cat_lgbm_0828_111.csv', index=False)